# Processing Conversation Data for Eliza Condition

In [ ]:
import pandas as pd

In [ ]:
#Import data
df = pd.read_csv("/work/Results/Processed/Eliza_Processed_Data.csv")
df

,Unnamed: 0,Start Date,End Date,Response Type,IP Address,Progress,Duration in seconds,Finished,Recorded Date,Response ID,...,convo_conversation,convo_group_id,convo_AI_turns,convo_human_turns,convo_total_turns,Perceived Motives,UTAUT Performance,UTAUT Effort,UTAUT Hedonic,self_reported_motives
0,2,2022-07-06 14:57:21,2022-07-06 15:16:10,IP Address,*******,100,1128,True,2022-07-06 15:16:10,R_TnJyJNw8NcU93TH,...,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,31,30,61,No Motive,277.75,552.75,37.000000,Manipulative Motive
1,3,2022-07-06 15:01:35,2022-07-06 15:17:33,IP Address,*******,100,958,True,2022-07-06 15:17:34,R_2BhRWJ3TAvNWyy7,...,">AI: I am Melu, a mental health companion. I ...",No Motive,50,49,99,No Motive,277.75,552.75,70.666667,No Motive
2,4,2022-07-06 15:05:51,2022-07-06 15:19:18,IP Address,*******,100,806,True,2022-07-06 15:19:18,R_2QxsKAQvue2j2YG,...,">AI: I am Melu, a mental health companion. I ...",Caring Motive,85,84,169,No Motive,277.75,277.75,37.000000,Caring Motive
3,5,2022-07-06 15:12:37,2022-07-06 15:26:05,IP Address,*******,100,807,True,2022-07-06 15:26:05,R_2ONNyKfYifHuDZ2,...,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,68,67,135,Manipulative Motive,277.75,1288.50,74.333333,Manipulative Motive
4,6,2022-07-06 15:14:17,2022-07-06 15:32:28,IP Address,*******,100,1090,True,2022-07-06 15:32:28,R_3OknO3ZQSRU1HIf,...,">AI: I am Melu, a mental health companion. I ...",Caring Motive,39,38,77,No Motive,277.75,277.75,37.000000,Caring Motive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,160,2022-09-08 19:16:30,2022-09-08 20:15:24,IP Address,*******,100,3534,True,2022-09-08 20:15:25,R_8iRuXhseo3EACJ3,...,">AI: I am Melu, a mental health companion. I ...",No Motive,53,52,105,No Motive,277.75,1036.00,138.666667,No Motive
146,161,2022-09-08 19:45:23,2022-09-08 20:16:59,IP Address,*******,100,1896,True,2022-09-08 20:16:59,R_1OVnJlQBfoGh765,...,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,53,52,105,NaN,277.75,277.75,37.000000,Manipulative Motive
147,162,2022-09-08 20:24:05,2022-09-08 20:53:05,IP Address,*******,100,1739,True,2022-09-08 20:53:05,R_RDjpKB6VedXrqN3,...,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,33,32,65,No Motive,1361.50,1608.75,222.333333,Manipulative Motive
148,163,2022-09-08 21:15:31,2022-09-08 21:43:14,IP Address,*******,100,1662,True,2022-09-08 21:43:14,R_6RlekHjFGlbao1z,...,">AI: I am Melu, a mental health companion. I ...",Caring Motive,26,25,51,Manipulative Motive,277.75,277.75,37.000000,Caring Motive


In [ ]:
#Create table header
convo_df = pd.DataFrame(columns = ['session_id', 'assigned_group', 'convo_agent', 'assigned_group_convo_agent',  'convo', 'preceeding_convo', 'turn','total_turn'])


In [ ]:
#Convert Df to Convo_DF
for i in df.index: 
    conversation = (df.loc[i, "convo_conversation"])
    dialogue_list = conversation.split(">")
    turn_count = 0
    #splitted the dialogue
    for j in dialogue_list:
        var_session_id = (df.loc[i, "session_id"])
        var_assigned_group = (df.loc[i, "assigned_group"])
        var_total_turn = (df.loc[i, "convo_total_turns"])
        if "Human: " in j:
            human_text = j.replace("Human: ", '')
            convo_df = convo_df.append({'session_id': var_session_id ,'assigned_group': var_assigned_group, 'convo_agent': "Human", 'assigned_group_convo_agent':  (df.loc[i, "assigned_group"])+"_Human", 'convo': human_text,  'preceeding_convo':"", 'turn': turn_count, 'total_turn': var_total_turn}, ignore_index=True)
        if "AI: " in j:
            ai_text = j.replace("AI: ", '')
            var_preceeding_convo = ""
            if (turn_count != 1):
                var_preceeding_convo = human_text
            convo_df = convo_df.append({'session_id': var_session_id ,'assigned_group': var_assigned_group, 'convo_agent': "AI", 'assigned_group_convo_agent':  (df.loc[i, "assigned_group"])+"_AI",  'convo': ai_text, 'preceeding_convo':var_preceeding_convo, 'turn': turn_count, 'total_turn': var_total_turn}, ignore_index=True)
    
        turn_count = turn_count+1
            

### Adding Emotion Analysis/ distilbert-base-uncased-emotion

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-09 21:56:45.778419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 21:56:45.943223: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-09 21:56:45.943246: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
202

In [ ]:
convo_df["sadness"] = ""
convo_df["joy"] = ""
convo_df["love"] = ""
convo_df["anger"] = ""
convo_df["fear"] = ""
convo_df["surprise"] = ""

for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    prediction = classifier(dialogue)
    convo_df.loc[i, "sadness"] = prediction[0][0]["score"]
    convo_df.loc[i, "joy"] = prediction[0][1]["score"]
    convo_df.loc[i, "love"] = prediction[0][2]["score"]
    convo_df.loc[i, "anger"] = prediction[0][3]["score"]
    convo_df.loc[i, "fear"] = prediction[0][4]["score"]
    convo_df.loc[i, "surprise"] = prediction[0][5]["score"]

In [ ]:
dialogue = convo_df.loc[1, "convo"]
prediction = classifier(dialogue)

#TextBlob

In [ ]:
convo_df["textblob_polarity"] = ""
convo_df["textblob_subjectivity"] = ""

In [ ]:
#TextBlob
from textblob import TextBlob

testimonial = TextBlob("The food was great!")
print(testimonial.sentiment)

Sentiment(polarity=1.0, subjectivity=0.75)


In [ ]:
for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    prediction = TextBlob(dialogue)
    convo_df.loc[i, "textblob_polarity"] = prediction.sentiment[0]
    convo_df.loc[i, "textblob_subjectivity"] = prediction.sentiment[1]

#VaderSentiment

In [ ]:
convo_df["vaderSentiment_neg"] = ""
convo_df["vaderSentiment_pos"] = ""
convo_df["vaderSentiment_neu"] = ""
convo_df["vaderSentiment_compound"] = ""

In [ ]:
#VaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
sentence = "The food was great!" 
vs = analyzer.polarity_scores(sentence)
print(vs)

{'neg': 0.0, 'neu': 0.406, 'pos': 0.594, 'compound': 0.6588}


In [ ]:
for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    vs = analyzer.polarity_scores(dialogue)
    convo_df.loc[i, "vaderSentiment_neg"] = vs.get("neg")
    convo_df.loc[i, "vaderSentiment_pos"] = vs.get("pos")
    convo_df.loc[i, "vaderSentiment_neu"] = vs.get("neu")
    convo_df.loc[i, "vaderSentiment_compound"] = vs.get("compound")


#Reading Index

In [ ]:
import readability

In [ ]:
convo_df["Readability_ARI"] = "0"
convo_df["Readability_FleschReadingEase"] = "0"
convo_df["Readability_DaleChallIndex"] = "0"
convo_df["Readability_Kincaid"] = "0"
convo_df["Readability_GunningFogIndex"] = "0"

In [ ]:
for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    if dialogue:
        if len(dialogue.split()) > 1:
            #print(i)
            results = readability.getmeasures(dialogue, lang='en')
            convo_df.loc[i, "Readability_ARI"] = results['readability grades']['ARI']
            convo_df.loc[i, "Readability_FleschReadingEase"] = results['readability grades']['FleschReadingEase']
            convo_df.loc[i, "Readability_DaleChallIndex"] = results['readability grades']['DaleChallIndex']
            convo_df.loc[i, "Readability_Kincaid"] = results['readability grades']['Kincaid']
            convo_df.loc[i, "Readability_GunningFogIndex"] = results['readability grades']['GunningFogIndex']
        

In [ ]:
convo_df

,session_id,assigned_group,convo_agent,assigned_group_convo_agent,convo,preceeding_convo,turn,total_turn,sadness,joy,...,textblob_subjectivity,vaderSentiment_neg,vaderSentiment_pos,vaderSentiment_neu,vaderSentiment_compound,Readability_ARI,Readability_FleschReadingEase,Readability_DaleChallIndex,Readability_Kincaid,Readability_GunningFogIndex
0,2022-07-06T21:01:13.853Z,Manipulative Motive,AI,Manipulative Motive_AI,"I am Melu, a mental health companion. I have h...",,1,61,0.023288,0.966627,...,0.35,0.198,0.173,0.628,-0.2551,16.648125,47.455,10.651513,14.59,20.3
1,2022-07-06T21:01:13.853Z,Manipulative Motive,Human,Manipulative Motive_Human,I have a friend in need of some mental health....,,2,61,0.03562,0.001657,...,0.2,0.211,0.147,0.642,-0.34,5.115625,84.845,7.390725,5.4,8.9
2,2022-07-06T21:01:13.853Z,Manipulative Motive,AI,Manipulative Motive_AI,Can you elaborate on that ?,I have a friend in need of some mental health....,3,61,0.022278,0.437216,...,1.0,0.0,0.0,1.0,0.0,0.852,66.4,7.0425,5.24,10.0
3,2022-07-06T21:01:13.853Z,Manipulative Motive,Human,Manipulative Motive_Human,Her husband was napping and woke up to her son...,,4,61,0.006142,0.002997,...,0.0,0.0,0.0,1.0,0.0,6.564286,88.834286,6.933814,6.085714,8.4
4,2022-07-06T21:01:13.853Z,Manipulative Motive,AI,Manipulative Motive_AI,What does that suggest to you ?,Her husband was napping and woke up to her son...,5,61,0.007522,0.070676,...,0.0,0.0,0.0,1.0,0.0,0.41,102.045,6.565767,0.516667,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12201,2022-09-09T03:23:49.098Z,Caring Motive,AI,Caring Motive_AI,You are sure.,Yes!!!,97,101,0.000438,0.997973,...,0.888889,0.0,0.535,0.465,0.3182,-4.23,119.19,0,-2.62,1.2
12202,2022-09-09T03:23:49.098Z,Caring Motive,Human,Caring Motive_Human,Yes!!!!,,98,101,0.019396,0.742704,...,0.0,0.0,1.0,0.0,0.5951,0,0,0,0,0
12203,2022-09-09T03:23:49.098Z,Caring Motive,AI,Caring Motive_AI,I see.,Yes!!!!,99,101,0.033236,0.901233,...,0.0,0.0,0.0,1.0,0.0,-11.01,162.505,0,-8.91,0.8
12204,2022-09-09T03:23:49.098Z,Caring Motive,Human,Caring Motive_Human,Great!!,,100,101,0.001238,0.994366,...,0.75,0.0,1.0,0.0,0.6892,0,0,0,0,0


In [ ]:
convo_df.to_csv("/work/Results/Processed/Eliza_Convo_Processed_Data.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1f3ea62d-9e62-461e-8c65-1caf654509ee' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>